| Etapa                      | Tecnologia recomendada                              | Justificativa                                           |
| -------------------------- | --------------------------------------------------- | ------------------------------------------------------- |
| Orquestração mensal        | `cron` no App Service ou Azure Functions            | Automação simples                                       |
| Ambiente isolado           | Docker                                              | Facilita dependências (`duckdb`, `pandas`, `azure-sdk`) |
| Extração + Upload Bronze   | Python puro (`requests`, `zipfile`, `DataLake SDK`) | Você já faz isso bem                                    |
| Transformações Silver/Gold | `Pandas` ou `DuckDB`                                | Leves, simples, ideais para CSV                         |
| Deploy                     | Azure App Service + GitHub Actions                  | CI/CD automatizado                                      |


cnes-data-pipeline/
│
├── Dockerfile
├── requirements.txt
├── app.py                  ← ponto de entrada principal
├── extract/
│   └── extract_cnes.py     ← download e bronze upload
├── transform/
│   ├── silver.py           ← limpeza e joins
│   └── gold.py             ← agregações e finais
├── utils/
│   └── azure_datalake.py   ← funções para leitura/escrita
└── cron/
    └── crontab.txt         ← para rodar mensalmente no container


In [2]:
from azure.storage.filedatalake import DataLakeServiceClient
import os

# Conexão com a conta
account_name = "cnesstorage"
account_key = "/ae47eZuE0NGPopxVHEkxOKsQwtEm3qQM0vBRPBRbB5nAW1zO6FPkEO9gwNQwkqExaVhOyHWgb68+AStIau+Uw=="#os.environ["AZURE_STORAGE_KEY"]
file_system_name = "gold"

# Criar cliente
service_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net",
    credential=account_key
)

file_system_client = service_client.get_file_system_client(file_system_name)

# Exibir apenas os arquivos de um mês

for path in file_system_client.get_paths(path=""):
    print(path.name)



estabelecimentos_202501_202507.parquet
estabelecimentos_all.parquet
estabelecimentos_some_periods.parquet
populacao.csv


In [4]:
import pandas as pd
from io import BytesIO

# Nome do arquivo (coloque o que você viu acima)
remote_path = "estabelecimentos_202501_202507.parquet"

# Cliente do arquivo
file_client = file_system_client.get_file_client(remote_path)

# Download em memória
download = file_client.download_file()
downloaded_bytes = download.readall()

# Ler com pandas
df = pd.read_parquet(BytesIO(downloaded_bytes))
df.head()


,CO_UNIDADE,CO_PROFISSIONAL_SUS,NO_PROFISSIONAL,CO_CBO,TP_SUS_NAO_SUS,DS_ATIVIDADE_PROFISSIONAL,NO_FANTASIA,NO_BAIRRO,NO_MUNICIPIO,CO_MUNICIPIO,CO_SIGLA_ESTADO,CO_CEP,ds_localidade,SK_REGISTRO,DATA_INGESTAO,yyyymm
0,SP00003509205000001329730000101,9D27061F6644A854,MARTHA TIDORI KIOTA KOTSUBO,225320,S,MEDICO EM RADIOLOGIA E DIAGNOSTICO POR IMAGEM,CLINICA MEDICA SULLA PELLE S C LTDA,ACLIMACAO,SAO PAULO,355030,SP,01530000,"01530000,SAO PAULO,SP,Brasil",SP00003509205000001329730000101_9D27061F6644A8...,2025-09-03,202501
1,SP00003509205000001329730000101,F3575C9617F8998A,WLADMIR GUBEISSI PINTO FILHO,225250,S,MEDICO GINECOLOGISTA E OBSTETRA,CLINICA MEDICA SULLA PELLE S C LTDA,ACLIMACAO,SAO PAULO,355030,SP,01530000,"01530000,SAO PAULO,SP,Brasil",SP00003509205000001329730000101_F3575C9617F899...,2025-09-03,202501
2,3500100047406,F6965A7E959C6A39,GRAZIELE DAVID,251510,N,PSICOLOGO CLINICO,GRAZIELE DAVID PSICOLOGA EIRELI,CENTRO,ADAMANTINA,350010,SP,17800037,"17800037,ADAMANTINA,SP,Brasil",3500100047406_F6965A7E959C6A39_251510,2025-09-03,202501
3,3500100081655,1FC46A4EA7312E5B,KELLY PRESTES RUFINO,223605,N,FISIOTERAPEUTA GERAL,RUFINO PRESTES LTDA,CENTRO,ADAMANTINA,350010,SP,17800057,"17800057,ADAMANTINA,SP,Brasil",3500100081655_1FC46A4EA7312E5B_223605,2025-09-03,202501
4,3500100109789,EDEB3090A41EB3A5,JOANA DARC BORRO,251510,N,PSICOLOGO CLINICO,JOANA DARC BORRO,VILA CICMA,ADAMANTINA,350010,SP,17803116,"17803116,ADAMANTINA,SP,Brasil",3500100109789_EDEB3090A41EB3A5_251510,2025-09-03,202501


## Métricas
- Relação profissionais / população
- Leitos hospitalares por habitante
- Cobertura da Atenção Básica -> Proporção da população vinculada a equipes de saúde da família

OBS: quebrar escopo para SP, olhar 

# Especialidades
MEDICO CARDIOLOGISTA -> 225120
MEDICO CLINICO -> 225125
MEDICO GERIATRA -> 225180
MEDICO PEDIATRA -> 225124
MEDICO DA ESTRATEGIA DE SAUDE DA FAMILIA -> 225142
MEDICO GINECOLOGISTA E OBSTETRA -> 225250
MEDICO NEUROLOGISTA -> 225112

# População

In [13]:
import re

year_cols = [c for c in df_pop.columns if re.fullmatch(r"\d{4}", str(c))]

df_pop_long = df_pop.melt(
    id_vars=["CO_MUNICIPIO", "NO_MUNICIPIO"],
    value_vars=year_cols,
    var_name="YYYY",
    value_name="POPULACAO"
)
df_pop_long

,CO_MUNICIPIO,NO_MUNICIPIO,YYYY,POPULACAO
0,15,Alta Floresta D'Oeste,2011,24.228
1,23,Ariquemes,2011,91.57
2,31,Cabixi,2011,6.221
3,49,Cacoal,2011,78.958
4,56,Cerejeiras,2011,16.939
...,...,...,...,...
72418,22005,Vianópolis,2025,15.644
72419,22054,Vicentinópolis,2025,9.175
72420,22203,Vila Boa,2025,4.145
72421,22302,Vila Propício,2025,6.028


# Médicos a cada 1000 habitantes

In [15]:
import re
import unicodedata
import pandas as pd
import pandasql as ps
from thefuzz import process

# ----------------------------
# Helpers
# ----------------------------
def norm_city(x: str) -> str:
    """Remove acentos, trim e upper para normalizar município."""
    if pd.isna(x):
        return x
    s = str(x)
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    return s.strip().upper()

def wide_pop_to_long_monthly(df_pop_wide: pd.DataFrame) -> pd.DataFrame:
    """
    Converte df_pop wide (colunas '2011'..'2025') para long (YYYY, POPULACAO)
    e replica para MM = 01..12 (população anual constante por mês).
    Espera colunas id: CO_MUNICIPIO, NO_MUNICIPIO.
    """
    # identifica colunas-ano (quatro dígitos)
    year_cols = [c for c in df_pop_wide.columns if re.fullmatch(r"\d{4}", str(c))]

    # derrete para vertical
    df_long = df_pop_wide.melt(
        id_vars=["CO_MUNICIPIO", "NO_MUNICIPIO"],
        value_vars=year_cols,
        var_name="YYYY",
        value_name="POPULACAO"
    )

    # limpa separadores (ex.: "24.228" -> 24228)
    df_long["POPULACAO"] = (
        df_long["POPULACAO"]
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df_long["POPULACAO"] = pd.to_numeric(df_long["POPULACAO"], errors="coerce")

    # normaliza município
    df_long["MUNICIPIO_NORM"] = df_long["NO_MUNICIPIO"].map(norm_city)

    # replica para meses 01..12 (cross join)
    months = pd.DataFrame({"MM": [f"{m:02d}" for m in range(1, 13)]})
    df_monthly = df_long.merge(months, how="cross")

    # mantém o essencial
    df_monthly = df_monthly[[
        "CO_MUNICIPIO", "NO_MUNICIPIO", "MUNICIPIO_NORM", "YYYY", "MM", "POPULACAO"
    ]].dropna(subset=["YYYY", "MM", "POPULACAO"])

    return df_monthly

def fuzzy_best_match(value: str, choices: list[str], threshold: int = 90) -> str | None:
    if pd.isna(value):
        return None
    match, score = process.extractOne(value, choices)
    return match if score >= threshold else None

# ----------------------------
# 1) Carregar população (wide) e converter para monthly long
# ----------------------------
df_pop_wide = pd.read_csv("/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/csv/populacao.csv")
df_pop_monthly = wide_pop_to_long_monthly(df_pop_wide)

# ----------------------------
# 2) Filtrar df base e quebrar yyyymm em YYYY/MM
#    (assume que 'df' já existe na sessão com a coluna yyyymm)
# ----------------------------
df_estab = df.query(
    "CO_CBO in ('225120','225125','225180','225124','225142','225250','225112') "
    " & NO_MUNICIPIO in ('INDAIATUBA','SAO CARLOS')"
).copy()

df_estab = df_estab.assign(
    YYYY=df_estab["yyyymm"].astype(str).str[:4],
    MM=df_estab["yyyymm"].astype(str).str[-2:]
)

# ----------------------------
# 3) Agregar profissionais por município/atividade/ano/mês
#    (usando pandasql como você vinha fazendo)
# ----------------------------
df_estab_grouped = ps.sqldf("""
SELECT
  NO_MUNICIPIO,
  DS_ATIVIDADE_PROFISSIONAL,
  YYYY,
  MM,
  COUNT(DISTINCT CO_PROFISSIONAL_SUS) AS TOTAL_PROFISSIONAIS
FROM df_estab
GROUP BY
  NO_MUNICIPIO, DS_ATIVIDADE_PROFISSIONAL, YYYY, MM
""", locals())

# normaliza município para o join
df_estab_grouped["MUNICIPIO_NORM"] = df_estab_grouped["NO_MUNICIPIO"].map(norm_city)

# ----------------------------
# 4) Join EXATO com população (MUNICIPIO_NORM + YYYY + MM)
# ----------------------------
join_keys = ["MUNICIPIO_NORM", "YYYY", "MM"]
df_join = df_estab_grouped.merge(
    df_pop_monthly[["MUNICIPIO_NORM", "YYYY", "MM", "POPULACAO"]],
    on=join_keys,
    how="left"
)

# ----------------------------
# 5) Fuzzy matching APENAS para os que ficaram sem população
# ----------------------------
missing_mask = df_join["POPULACAO"].isna()
if missing_mask.any():
    choices = df_pop_monthly["MUNICIPIO_NORM"].dropna().unique().tolist()

    df_unmatched = df_join.loc[missing_mask].copy()
    df_unmatched["MUNICIPIO_NORM_FUZZY"] = df_unmatched["MUNICIPIO_NORM"].apply(
        lambda v: fuzzy_best_match(v, choices, threshold=90)
    )

    # re-merge usando o nome fuzzy
    df_unmatched = df_unmatched.merge(
        df_pop_monthly[["MUNICIPIO_NORM", "YYYY", "MM", "POPULACAO"]],
        left_on=["MUNICIPIO_NORM_FUZZY", "YYYY", "MM"],
        right_on=["MUNICIPIO_NORM", "YYYY", "MM"],
        how="left",
        suffixes=("", "_FZ")
    )

    # preenche POPULACAO faltante com a fuzzy quando disponível
    df_join.loc[missing_mask, "POPULACAO"] = df_unmatched["POPULACAO_FZ"].values

# ----------------------------
# 6) Métrica final: profissionais por 1.000 habitantes
# ----------------------------
df_final = df_join.copy()
df_final["PROFISSIONAIS_POR_1000"] = (
    df_final["TOTAL_PROFISSIONAIS"] / df_final["POPULACAO"]
) * 1000

# colunas finais e ordenação
df_final = df_final[
    [
        "NO_MUNICIPIO",
        "DS_ATIVIDADE_PROFISSIONAL",
        "YYYY",
        "MM",
        "TOTAL_PROFISSIONAIS",
        "POPULACAO",
        "PROFISSIONAIS_POR_1000",
    ]
].sort_values(
    ["NO_MUNICIPIO", "DS_ATIVIDADE_PROFISSIONAL", "YYYY", "MM"],
    ignore_index=True
)

# visualize
df_final


,NO_MUNICIPIO,DS_ATIVIDADE_PROFISSIONAL,YYYY,MM,TOTAL_PROFISSIONAIS,POPULACAO,PROFISSIONAIS_POR_1000
0,INDAIATUBA,MEDICO CARDIOLOGISTA,2025,01,47,269657.0,0.174295
1,INDAIATUBA,MEDICO CARDIOLOGISTA,2025,02,47,269657.0,0.174295
2,INDAIATUBA,MEDICO CARDIOLOGISTA,2025,04,47,269657.0,0.174295
3,INDAIATUBA,MEDICO CARDIOLOGISTA,2025,06,48,269657.0,0.178004
4,INDAIATUBA,MEDICO CARDIOLOGISTA,2025,07,48,269657.0,0.178004
...,...,...,...,...,...,...,...
100,SAO CARLOS,MEDICO PEDIATRA,2025,04,116,10462.0,11.087746
101,SAO CARLOS,MEDICO PEDIATRA,2025,06,118,266427.0,0.442898
102,SAO CARLOS,MEDICO PEDIATRA,2025,06,118,10462.0,11.278914
103,SAO CARLOS,MEDICO PEDIATRA,2025,07,117,266427.0,0.439145


In [23]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.9 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata

# ---- helpers ----
def _norm_city(x: str) -> str:
    if pd.isna(x):
        return x
    s = unicodedata.normalize("NFKD", str(x)).encode("ASCII", "ignore").decode("ASCII")
    return s.strip().upper()

def _make_date(df: pd.DataFrame) -> pd.Series:
    yyyy = df["YYYY"].astype(int).astype(str)
    mm   = df["MM"].astype(int).astype(str).str.zfill(2)
    return pd.to_datetime(yyyy + "-" + mm + "-01")

def plot_metric_by_specialty(
    df_final: pd.DataFrame,
    city: str,
    metric: str = "PROFISSIONAIS_POR_1000",  # ou "TOTAL_PROFISSIONAIS"
    top_n: int = 8,                            # limita as top N especialidades (por média recente)
    start_yyyymm: str | None = None,          # ex.: "202401" para filtrar a partir de jan/2024
    end_yyyymm: str | None = None,            # ex.: "202507" para filtrar até jul/2025
    rolling: int | None = None                # janela de média móvel (ex.: 3 para 3 meses)
):
    # validações básicas
    required = {"NO_MUNICIPIO", "DS_ATIVIDADE_PROFISSIONAL", "YYYY", "MM", metric}
    missing = required - set(df_final.columns)
    if missing:
        raise ValueError(f"df_final está faltando colunas: {missing}")

    # normaliza cidade e filtra
    city_norm = _norm_city(city)
    df = df_final.copy()
    df["MUNICIPIO_NORM"] = df["NO_MUNICIPIO"].apply(_norm_city)
    df = df[df["MUNICIPIO_NORM"] == city_norm].copy()
    if df.empty:
        print(f"Nenhum dado para a cidade '{city}'.")
        return

    # cria coluna de data
    df["date"] = _make_date(df)

    # recorte por período (opcional)
    if start_yyyymm:
        s_y, s_m = int(str(start_yyyymm)[:4]), int(str(start_yyyymm)[-2:])
        df = df[(df["YYYY"].astype(int) > s_y) | ((df["YYYY"].astype(int) == s_y) & (df["MM"].astype(int) >= s_m))]
    if end_yyyymm:
        e_y, e_m = int(str(end_yyyymm)[:4]), int(str(end_yyyymm)[-2:])
        df = df[(df["YYYY"].astype(int) < e_y) | ((df["YYYY"].astype(int) == e_y) & (df["MM"].astype(int) <= e_m))]

    if df.empty:
        print("Sem dados após aplicar os filtros de período.")
        return

    # escolhe top N especialidades pelo valor médio (nos últimos 12 meses se possível)
    df_sorted = df.sort_values("date")
    cutoff = df_sorted["date"].max() - pd.DateOffset(months=12)
    recent = df_sorted[df_sorted["date"] >= cutoff] if (df_sorted["date"].min() < cutoff) else df_sorted
    top_specialties = (
        recent.groupby("DS_ATIVIDADE_PROFISSIONAL")[metric]
        .mean()
        .sort_values(ascending=False)
        .head(top_n)
        .index.tolist()
    )

    df_top = df_sorted[df_sorted["DS_ATIVIDADE_PROFISSIONAL"].isin(top_specialties)].copy()

    # (opcional) média móvel
    if rolling and rolling > 1:
        df_top[metric] = (
            df_top.sort_values(["DS_ATIVIDADE_PROFISSIONAL", "date"])
                  .groupby("DS_ATIVIDADE_PROFISSIONAL")[metric]
                  .transform(lambda s: s.rolling(rolling, min_periods=1).mean())
        )

    # plot
    plt.figure(figsize=(12, 6))
    for esp, g in df_top.groupby("DS_ATIVIDADE_PROFISSIONAL"):
        g = g.sort_values("date")
        plt.plot(g["date"], g[metric], label=esp)

    plt.title(f"{metric.replace('_', ' ').title()} — {city_norm}")
    plt.xlabel("Data")
    plt.ylabel(metric.replace("_", " ").title())
    plt.legend(title="Especialidade", ncol=2, fontsize=9)
    plt.tight_layout()
    plt.show()
